In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
#from tensorflow.python.ops import rnn, rnn_cell
from tensorflow.contrib import rnn

In [ ]:
mnist = input_data.read_data_sets("/tmp/data", one_hot = True)

In [ ]:
'''
In machine learning, an epoch is a full iteration over samples. Here, we are restricting the model
to 10 complete epochs or cycles of the algorithm running through the dataset.

The batch variable determines the amount of data being fed to the algorithm
at any given time, in this case, 100 images.
'''
n_classes = 10
batch_size = 128
hm_epochs = 3

chunk_size = 28 #28 cols
n_chunks = 28 #28 rows
rnn_size = 128

# Height x Weight matrix
# None represents batch size

'''
The method tf.placeholder allows us to create variables that act as nodes holding the data.
Here, x is a 2-dimensionall array holding the MNIST images, with none implying the batch size
(which can be of any size) and 784 being a single 28×28 image. y is the target output class that
consists of a 2-dimensional array of 10 classes (denoting the numbers 0-9) that identify what digit is stored in each image.

'''
x = tf.placeholder('float',[None, n_chunks, chunk_size])
y = tf.placeholder('float',[None,n_classes]) #label for x

In [ ]:
def recurrent_neural_network(x):
    layer = {'weights': tf.Variable(tf.random_normal([rnn_size, n_classes])),
                      'biases': tf.Variable(tf.random_normal([n_classes]))}
    
    x = tf.transpose(x,[1,0,2])
    x = tf.reshape(x, [-1, chunk_size])
    x = tf.split(x, n_chunks, 0)
    
    lstm_cell = rnn.BasicLSTMCell(rnn_size)
    
    '''
    The output generated by static_rnn is a list of tensors of shape [batch_size,num_units].
    The length of the list is number of time steps through which network is unrolled
    i.e. one output tensor for each time step
    
    '''
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype = tf.float32)
    
    output = tf.matmul(outputs[-1], layer['weights']) + layer['biases']
    
    return output
    

In [ ]:
''' We will be using a simple softmax model to implement our network. Softmax is a generalization of logistic regression,
usually used in the final layer of a network. It is useful because it helps in multi-classification models where a given
output can be a list of many different things.
It provides values between 0 to 1 that in addition give you the probability of the output belonging to a particular class. 
'''

def train_neural_network(x):
    prediction = recurrent_neural_network(x)

    ''' This is the cost function of the model – a cost function is a difference between the predicted value and
    the actual value that we are trying to minimize to improve the accuracy of the model'''
    
    cost  = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = prediction,labels = y)) #loss
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        for epoch in range(hm_epochs):
            epoch_loss = 0
            for _ in range(int(mnist.train.num_examples/batch_size)):
                epoch_x, epoch_y = mnist.train.next_batch(batch_size)
                epoch_x = epoch_x.reshape((batch_size, n_chunks, chunk_size))
                _,c = sess.run([optimizer,cost], feed_dict= {x: epoch_x, y: epoch_y}) #c is loss
                
                epoch_loss += c
            print('Epoch ',epoch+1,' completed out of ',hm_epochs,', loss ',epoch_loss)
            
        correct = tf.equal(tf.argmax(prediction,1), tf.argmax(y,1))
        accuracy  = tf.reduce_mean(tf.cast(correct,'float'))
        
        print('Accuracy: ',accuracy.eval({ x: mnist.test.images.reshape((-1, n_chunks, chunk_size)), y: mnist.test.labels }))
        

In [ ]:
# hm_epochs = 4, accuracy = 93%
# hm_epochs = 10, accuracy = 95.14%
#hm_epochs = 15, accuracy = 95.54%
#hm_epochs = 10, accuracy = 98.29% with RNN
#hm_epochs = 3, accuracy = 97.38% with RNN
#tf.reset_default_graph() , try to add code in single file 
train_neural_network(x)

In [ ]:
tf.__version__